In [1]:
import requests
import cv2
import numpy as np
import contextlib
import os
import json
from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=False, lang='en',show_log=False,ocr_version="PP-OCRv3")
import moviepy.editor as mp
import whisper
model = whisper.load_model("base.en")
model2 = whisper.load_model("base")
import ast

In [ ]:
profili=["lilyebert","tovafriedman","thetrueadventures"]

In [12]:
def get_from_api(api,params,**kwargs):
    # domain='https://www.tikwm.com/'
    domain="https://tiktok-video-no-watermark2.p.rapidapi.com/"
    headers = {
        "X-RapidAPI-Key": "",
        "X-RapidAPI-Host": ""
    }
    res = requests.get(
    url=domain+api,
    headers = headers,
    params = params,
    **kwargs)
    try:
        if res.status_code==429: return (res,None)
        return (res, res.json()['data']) if res.json()['code'] == 0 else (res, None)
    except:
        print("log")
        with open("log.txt","a") as f:
            f.write(str(res.status_code))
            f.write(str(res.json()))
            f.write("")            
        return (None,None)

In [13]:
def get_video_info(url,**kwargs):
    return get_from_api('', params={"url": url,"web": 1, "hd": 1},**kwargs)

In [14]:
def download_no_watermark(url, filename):
    res,data=get_video_info(url)
    if data is None:
        return False
    url_no_watermark = 'https://www.tikwm.com' + data["play"]
    r = requests.get(url_no_watermark)
    with open(filename, 'wb') as outfile:
        outfile.write(r.content)
    return True

In [2]:
def get_saving_frames_durations(cap, saving_fps):
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    return list(np.arange(0, clip_duration, 1 / saving_fps))

def process(video,SAVING_FRAMES_PER_SECOND):
    cap=cv2.VideoCapture(video)  
    fps = cap.get(cv2.CAP_PROP_FPS)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    index = 0
    out=[]
    while True:
        is_read, frame = cap.read()
        if not is_read:
            break
        frame_duration = index / fps
        try:
            closest_duration = saving_frames_durations[0]
        except IndexError:
            break
        if frame_duration >= closest_duration:
            out.append(frame)
            with contextlib.suppress(IndexError):
                saving_frames_durations.pop(0)
        index += 1
    cap.release()
    cv2.destroyAllWindows()
    return out

In [ ]:
profili_analizzati=json.load(open("profili_analizzati.json"))

In [ ]:
for profilo,value in profili_analizzati.items():
    l_video=os.listdir(f"../Video/2022-12-09/{profilo}")
    for video_id in value["video"].keys():
        if f"{video_id}.mp4" not in l_video:
            if not download_no_watermark(video_id,f"video/{profilo}/{video_id}.mp4"):
                print(f"Errore nel download del video {video_id}")

In [50]:
lista_interessanti=json.load(open("lista_interessanti.txt","r"))
lista_interessanti_profilo_video=[(video.split("/")[-3][1:],video.split("/")[-1]) for video in lista_interessanti]

In [ ]:
for profilo,video in lista_interessanti_profilo_video:
    img_array=process(f"{profilo}/{video}.mp4",1)
    complete=[]
    for img in img_array:
        result = ocr.ocr(img, cls=False)
        result_tmp=[val2[-1][0] for val in result for val2 in val]
        complete.append(result_tmp)
    json.dump(complete,open(f"nuovi_video/{profilo}_{video}.txt","w"))

In [86]:
lista_ocr=[file for file in os.listdir() if file[-3:]=="txt"]

In [96]:
for file in lista_ocr:
    with open(file,"r") as f:
        lista=ast.literal_eval(f.read())
        testo_completo=[]
        for frase in lista:
            frase=" ".join(frase)
            if not testo_completo or frase!=testo_completo[-1]:
                testo_completo.append(frase)
        json.dump(testo_completo,open(f"testo_unito/{file}","w"))

In [ ]:
base="video/lilyebert"
lista_video=os.listdir(base)
lista_ocr=os.listdir(f"{base}/ocr")

for video in lista_video:
    filename_ocr=f"{video[:-4]}.txt"
    filetype=video[-3:]
    if filename_ocr in lista_ocr or filetype!="mp4":
        continue
    img_array=process(f"{base}/{video}",1)
    complete=[]
    for img in img_array:
        result = ocr.ocr(img, cls=True)
        text= " ".join(val2[-1][0] for val in result for val2 in val)
        if text not in complete:
            complete.append(text)
    json.dump(complete,open(f"{base}/ocr/{filename_ocr}","w"))
    lista_ocr.append(filename_ocr)

In [ ]:
for p in profili:
    video_pa=[v for v in os.listdir(p) if v[-3:]=="mp4"]
    for v in video_pa:
        videoTmp=mp.VideoFileClip(f"{p}/{v}")
        videoTmp.audio.write_audiofile(f"{p}/audio/{v[:-3]}mp3",logger=None, verbose=False)

In [24]:
profilo_audio=[(stringa.split("/")[-3][1:],f"{stringa.split('/')[-1]}.mp3") for stringa in lista_interessanti]

In [ ]:
for profilo,audio in profilo_audio:
    result = model.transcribe(f"{profilo}/audio/{audio}",fp16=False)
    json.dump(result["text"],open(f"{profilo}/audio/testo/{audio[:-3]}txt","w"))